# Analysing Sentinel 5p data with Google Earth Engine

## Prerequisites

1. Create an account on google earth engine. Open the link https://earthenginegoogle.com/signup/ or click Sign up in the upper right
corner of the Earth Engine Website (https://earthengine.google.com/).
2. View this tutorial to sign up https://www.youtube.com/watch?v=S0AzoP40cDI

## About Sentinel 5p

The Sentinel-5 Precursor mission instrument collects data useful for assessing air quality. The TROPOMI instrument is a multispectral sensor that records reflectance of wavelengths important for measuring atmospheric concentrations of ozone, methane, formaldehyde, aerosol, carbon monoxide, nitrogen oxide, and sulphur dioxide, as well as cloud characteristics at a spatial resolution of 0.01 arc degrees.


1. **Sentinel-5P UV Aerosol Index**

A measure of the prevalence of aerosols in the atmosphere. Ideal for tracking the evolution of episodic aerosol plumes from dust outbreaks, volcanic ash, and biomass burning. Dataset availability: 2018-07-04 - Present

2. **Sentinel-5P Carbon Monoxide**

Concentrations of Carbon monoxide (CO) and water vapor. CO is an important atmospheric trace gas for our understanding of tropospheric chemistry. Main sources of CO are combustion of fossil fuels, biomass burning, and atmospheric oxidation of methane and other hydrocarbons. Dataset availability: 2018-06-28 - Present

3. **Sentinel-5P Nitrogen Dioxide**

Total, tropospheric, and stratospheric nitrogen dioxide concentration. Nitrogen dioxide enters the atmosphere as a result of anthropogenic activities such as fossil fuel combustion and biomass burning, as well as natural processes including microbiological processes in soils, wildfires and lightning. Dataset availability: 2018-06-28 - Present

4. **Sentinel-5P Sulphur Dioxide**

Atmospheric sulphur dioxide (SO₂) concentration. SO₂ enters Earth’s atmosphere through both natural and anthropogenic processes, though the majority is of anthropogenic origin. SO₂ emissions adversely affect human health and air quality and also have an effect on climate through radiative forcing. Dataset availability: 2018-07-10 - Present

5. **Sentinel-5P Methane**

Atmospheric methane (CH₄) concentration. After carbon dioxide (CO₂), it is the most important contributor to the anthropogenically enhanced greenhouse effect. It enters Earth’s atmosphere through both natural and anthropogenic processes, though the majority is of anthropogenic origin. Dataset availability: 2019-02-08 – Present

6. **Sentinel-5P Ozone**

Total atmospheric column ozone concentration. Ozone shields the biosphere from solar ultraviolet radiation. In the troposphere, it acts as an efficient cleansing agent, but at high concentrations it also becomes harmful to the health of humans, animals, and vegetation. Ozone is also an important greenhouse-gas contributing to ongoing climate change. Dataset availability: 2018-07-10 – Present

7. **Sentinel-5P Formaldehyde**

Formaldehyde concentration. Formaldehyde is an intermediate gas in almost all oxidation chains of non-methane volatile organic compounds (NMVOC), leading eventually to CO₂. Sources include vegetation, fires, traffic and industrial Dataset availability: 2018-10-02 – Present

8. **Sentinel-5P Cloud**

Cloud characteristics including: fraction, height and pressure for base and top, optical depth, and surface albedo. Dataset availability: 2018-07-04 – Present

## Setup & Imports

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap
import os

In [ ]:
Map = geemap.Map()
Map

## Setup output directory

In [ ]:
out_dir = os.path.expanduser('./')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

## Add Shapefile

In [ ]:
# load shapefile from GEE assets
states = ee.FeatureCollection("users/saheel/state_shapefile")
Map.addLayer(states, {}, 'Indian States')


# load shapefile from local drive
states_shp = '../data/state_shapefile.shp'
states = geemap.shp_to_ee(states_shp)
Map.addLayer(states, {}, 'Indian States')

## Create an annual composite of Sentinel 5p Nitrogen Dioxide

In [ ]:
# define an empty map
Map = geemap.Map()

# load the sentinel 5p collection
# filter using start and end date
# load the parameters
# check all the parameters https://developers.google.com/earth-engine/datasets/catalog/sentinel-5p
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
                  .filter(ee.Filter.date('2022-01-01', '2022-12-31')) \
                  .select("tropospheric_NO2_column_number_density")\
                  .filterBounds(states)

# styling inputs
image_vis_params = {
  'min': 0,
  'max': 0.0002,
  'palette': ['#006698', '#3e8eb4', '#7db6d2', '#abb8be', '#b7be9e', '#d1a45b', '#dd7f3d', '#4b0c00']
}
boundary_vis_params = {
'fillColor': '#FF000000', 'width': 1, 'lineType': 'dashed'
}

# add the layers on the map
# create a annual mean composite and add it to the map
Map.addLayer(collection.mean().clip(states), image_vis_params, 'S5P NO2 2022')
Map.addLayer(states.style(**boundary_vis_params),{},'States Boundary')
Map.setCenter(78.96, 20.59, 5)
Map

## Extract Zonal Stats and save as csv file

In [ ]:
# save zonal stats from annual composite
s5p_no2 = collection.mean().clip(states)
out_s5p_stats = os.path.join("./", 's5p_india.csv')
geemap.zonal_statistics(s5p_no2, states, out_s5p_stats, statistics_type='MEAN', scale=1000)

In [ ]:
# save zonal stats for each day, it might take some time depending on amount of data.
s5p_no2 = collection.toBands()
out_s5p_stats = os.path.join("./", 's5p_india_daily.csv')
geemap.zonal_statistics(s5p_no2, states, out_s5p_stats, statistics_type='MEAN', scale=1000)